In [4]:
##before continuing the following packages need to be runned
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)
source('tests.R')
source('cleanup.R')

Warning message in file(filename, "r", encoding = encoding):
“cannot open file 'tests.R': No such file or directory”


ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


In [11]:
portfolio_data <- read_csv("final_data.csv")|>
mutate(max_portfolio_return_name = as_factor(max_portfolio_return_name))
portfolio_data

Rows: 25 Columns: 6
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): max_portfolio_return_name
dbl (5): year, annual_inflation, real_gdp_growth_rate_percentage, annual_une...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


year,annual_inflation,real_gdp_growth_rate_percentage,annual_unemployment_rate,max_portfolio_return_value,max_portfolio_return_name
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1997,2.337690,4.4,4.950000,22.37,60/40_portfolio_return_percentage
1998,1.552279,4.5,4.508333,18.26,growth_portfolio_return_percentage
1999,2.188027,4.8,4.216667,20.79,growth_portfolio_return_percentage
⋮,⋮,⋮,⋮,⋮,⋮
2019,1.812210,2.3,3.666667,23.59,growth_portfolio_return_percentage
2020,1.233584,-3.4,8.091666,15.74,growth_portfolio_return_percentage
2021,4.697859,5.7,5.366667,15.24,growth_portfolio_return_percentage


In [15]:
set.seed(9999) #ensuring reproducibility
options(repr.plot.height = 5, repr.plot.width = 6)

portfolio_split <- initial_split(portfolio_data, prop = 0.75, strata = max_portfolio_return_name)
portfolio_train <- training(portfolio_split)
portfolio_test <- testing(portfolio_split) 

In [34]:
knn_recipe <- recipe(max_portfolio_return_name ~ annual_inflation + real_gdp_growth_rate_percentage + annual_unemployment_rate, data = portfolio_train) |>
    step_center(all_predictors()) |>
    step_scale (all_predictors())

In [35]:
knn_tune <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |> 
set_engine("kknn") |>
set_mode("classification") 

In [36]:
knn_vfold <- vfold_cv(portfolio_train, v = 5, strata = max_portfolio_return_name)
gridvals <- tibble(neighbors = seq(from = 1, to = 20))

knn_results <- workflow() |>
  add_recipe(knn_recipe) |>
  add_model(knn_tune) |>
  tune_grid(resamples = knn_vfold, grid = gridvals) |>
  collect_metrics() 

! Fold1: preprocessor 1/1, model 1/1: 20 samples were requested but there were 13 rows in the data. 8 will be ...

! Fold1: preprocessor 1/1, model 1/1 (predictions): data length [60] is not a sub-multiple or multiple of the number of colu...

x Fold1: preprocessor 1/1, model 1/1 (predictions): Error in `[.default`(cl, C): only 0's may be mixed with negative subscripts

! Fold2: preprocessor 1/1, model 1/1: 20 samples were requested but there were 14 rows in the data. 9 will be ...

! Fold2: preprocessor 1/1, model 1/1 (predictions): data length [56] is not a sub-multiple or multiple of the number of colu...

x Fold2: preprocessor 1/1, model 1/1 (predictions): Error in `[.default`(cl, C): only 0's may be mixed with negative subscripts

! Fold3: preprocessor 1/1, model 1/1: 20 samples were requested but there were 14 rows in the data. 9 will be ...

! Fold3: preprocessor 1/1, model 1/1 (predictions): data length [52] is not a sub-multiple or multiple of the number of colu...

x Fold3: p